In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_excel("C:\\Users\\stadn\\Desktop\\_robot_dane_operator\\wyniki.xlsx")

In [8]:
df = df.drop([0], axis=0)

In [9]:
df["Nr obserwacji"] = np.arange(1, 31)

In [10]:
df.head()

,Nr obserwacji,Operator 1,Unnamed: 2,Operator 2,Unnamed: 4,Operator 3,Unnamed: 6,Operator 4,Unnamed: 8,Operator 5,Unnamed: 10
1,1,167,98,85,121,148,130,168,171,174,117
2,2,138,83,102,128,164,123,208,146,156,73
3,3,101,105,108,133,168,114,184,132,174,115
4,4,120,111,78,116,171,119,207,145,164,115
5,5,135,101,85,110,147,110,201,139,163,120


In [11]:
operator1 = df[["Nr obserwacji", "Operator 1", "Unnamed: 2"]]
operator1.columns = (["Nr obserwacji", "X", "Y"])

In [12]:
operator2 = df[["Nr obserwacji", "Operator 2", "Unnamed: 4"]]
operator2.columns = (["Nr obserwacji", "X", "Y"])

In [13]:
operator3 = df[["Nr obserwacji", "Operator 3", "Unnamed: 6"]]
operator3.columns = (["Nr obserwacji", "X", "Y"])

In [14]:
operator4 = df[["Nr obserwacji", "Operator 4", "Unnamed: 8"]]
operator4.columns = (["Nr obserwacji", "X", "Y"])

In [15]:
operator5 = df[["Nr obserwacji", "Operator 5", "Unnamed: 10"]]
operator5.columns = (["Nr obserwacji", "X", "Y"])

In [16]:
operators = [operator1, operator2, operator3, operator4, operator5]

In [17]:
def predict_y(data, test_size):
    X = data.drop("Y", axis=1)
    y = data["Y"]
    
    poly_converter = PolynomialFeatures(degree=3, include_bias=False)
    poly_converter.fit(X)
    poly_features = poly_converter.transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=test_size, random_state=42)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    pred_y = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, pred_y))
    
    
    
    return pred_y

In [18]:
def predict_x(data, test_size):
    X = data.drop("X", axis=1)
    y = data["X"]
    
    poly_converter = PolynomialFeatures(degree=3, include_bias=False)
    poly_converter.fit(X)
    poly_features = poly_converter.transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=test_size, random_state=42)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    pred_x = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, pred_x))
    
    
    
    return pred_x

In [19]:
sizes = [0.1, 0.16, 0.33]
pred_results = []
pred_results = pd.DataFrame(pred_results)
x = 1
for operator in operators:
    predict_y(operator, 0.2)
    pred_results[f"X{x}"] = predict_x(operator, 0.2)
    pred_results[f"Y{x}"] = predict_y(operator, 0.2)
    print([predict_x(operator, 0.2), predict_y(operator,0.2)])

[array([223.12274989, 172.32302086, 180.83528608, 184.45074928,
       157.20907594, 165.75194039]), array([109.03907151, 108.86235816, 110.99206522, 106.27053404,
       110.52188516, 112.37116021])]
[array([ 79.13185989,  98.62813821,  99.71171177, 112.5726261 ,
        93.46799222,  94.81932579]), array([ 88.63139524, 131.65273836, 104.65553401, 110.84897563,
       115.14155972, 113.62237152])]
[array([178.95305044, 136.16787898, 141.95734259, 157.1543096 ,
       158.66918395, 146.25358671]), array([ 39.33585636,  99.97293702,  94.74382695, 103.09173287,
        99.03244903, 101.15408048])]
[array([-226.13333968,  213.09275043,  228.89685416,  203.92519865,
        207.01068355,  210.50911527]), array([110.82479789, 134.78585839, 109.17744627, 128.92300231,
       145.09528083, 149.57184094])]
[array([180.9511832 , 163.40927513, 174.65923591, 167.54510011,
       158.53002357, 170.09955582]), array([100.5685526 , 100.36997653, 123.75857666, 101.41184389,
       108.91243394, 112.8

In [23]:
sizes = [0.1, 0.16, 0.33]
pred_results = []
pred_results = pd.DataFrame(pred_results)
x = 1
for operator in operators:
    for size in sizes:
        predict_y(operator, size)
        #pred_results[f"X{x}"] = predict_x(operator, size)
        #pred_results[f"Y{x}"] = predict_y(operator, size)
        #print([predict_x(operator, size), predict_y(operator,size)])
    


[array([226.43291757, 172.52008565, 179.06079401]), array([110.49952858, 109.76374085, 111.31448242])]
[array([225.23872356, 172.40320056, 180.53067338, 184.77767978,
       158.87748097]), array([108.60153458, 109.02816431, 110.93728085, 106.29506773,
       110.66478322])]
[array([242.20894473, 175.85389065, 176.502947  , 171.33193673,
       167.17262823, 172.25715796, 213.76506931, 183.59353942,
       177.70553418,  78.11555682]), array([113.33762571, 109.63806933, 105.9681561 , 111.96457477,
       100.13312455, 112.31976244, 135.22577485, 109.59120497,
       118.08972953,  15.12751196])]
[array([ 80.57267821,  97.10586038, 101.45787666]), array([ 88.77777673, 133.55650001, 104.6389071 ])]
[array([ 79.04677744,  98.54210077,  99.60382861, 112.81138037,
        93.74653049]), array([ 88.70894488, 131.92757799, 104.91529289, 110.48549512,
       114.04133031])]
[array([ 56.35844317, 135.98178886, 106.17110372, 123.36540313,
        76.41081979,  82.19578956, -24.71171098,  81.3839

In [58]:
pred_results.to_excel("C:\\Users\\stadn\\Desktop\\_robot_dane_operator\\predictions.xlsx")

In [59]:
def predict_y(data, regressor):
    X = data.drop("Y", axis=1)
    y = data["Y"]
    
    if regressor == "polynomial":
        from sklearn.preprocessing import PolynomialFeatures
        poly_converter = PolynomialFeatures(degree=3, include_bias=False)
        poly_converter.fit(X)
        poly_features = poly_converter.transform(X)
    
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.2, random_state=42)
    
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
        from sklearn.linear_model import LinearRegression
        model = LinearRegression()
        model.fit(X_train, y_train)
        pred_y = model.predict(X_test)
    
        from sklearn.metrics import mean_squared_error
        rmse = np.sqrt(mean_squared_error(y_test, pred_y))
        
    else:
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    
    if regressor == "linear":
        from sklearn.linear_model import LinearRegression
        model = LinearRegression()
        model.fit(X_train, y_train)
        pred_y = model.predict(X_test)
        
        from sklearn.metrics import mean_squared_error
        rmse = np.sqrt(mean_squared_error(y_test, pred_y))
        
    elif regressor == "svr":
        from sklearn.svm import SVR
        model = SVR()
        model.fit(X_train, y_train)
        pred_y = model.predict(X_test)
        
        from sklearn.metrics import mean_squared_error
        rmse = np.sqrt(mean_squared_error(y_test, pred_y))
        
    return rmse

In [65]:
regressors = ["polynomial", "linear", "svr"]
pred_results = []
pred_results = pd.DataFrame(pred_results)
x = 1
for operator in operators:
    for regressor in regressors:
        predict_y(operator, regressor)
        #pred_results[f"X{x}"] = predict_x(operator)
        #pred_results[f"Y{x}"] = predict_y(operator, regressor)
        print(predict_y(operator, regressor))
        x += 1

11.5745986027087
16.252748294883954
11.885675392032066
9.525612774487563
14.866398503167913
11.32021653365787
14.255617785020823
12.99188531927763
22.25508231435714
44.32485625786355
45.43807093137654
52.843177971674706
12.729119716365878
9.220055845594018
5.460220513297156


In [63]:
pred_results

""
